In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def load_kitti_bin(bin_path):
    point_cloud = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return point_cloud[:, :3]  # Ignore reflectance

def visualize_point_cloud(points, colors=None):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    if colors is not None:
        pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd])

In [ ]:
# raw lidar visualization

bin_path = "/home/julia/workspace/datasets/sequences/05/velodyne/000000.bin"  # Replace with a valid .bin file path

points = load_kitti_bin(bin_path)
visualize_point_cloud(points)


In [ ]:
# visualization after removing the ground

def remove_ground_by_height(points, z_threshold=-1.4):
    """
    Remove ground points using a Z-height threshold.
    Points below z_threshold are considered ground.
    """
    ground_mask = points[:, 2] < z_threshold
    non_ground_points = points[~ground_mask]
    ground_points = points[ground_mask]
    return non_ground_points, ground_points

def visualize_ground_removal(non_ground, ground):
    """Visualize ground and non-ground points in different colors."""
    pcd_ground = o3d.geometry.PointCloud()
    pcd_ground.points = o3d.utility.Vector3dVector(ground)
    pcd_ground.paint_uniform_color([0.5, 0.5, 0.5])  # Gray

    pcd_objects = o3d.geometry.PointCloud()
    pcd_objects.points = o3d.utility.Vector3dVector(non_ground)
    pcd_objects.paint_uniform_color([1, 0, 0])  # Red

    o3d.visualization.draw_geometries([pcd_ground, pcd_objects], window_name="Ground Removal")


In [15]:
# Load LiDAR points
bin_file = bin_path
points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)[:, :3]

# Apply ground removal
non_ground, ground = remove_ground_by_height(points)

In [ ]:
# ground = gray, non-ground = red

visualize_ground_removal(non_ground, ground)


In [ ]:
#object clustering using dbscan

def cluster_dbscan(points, eps=0.5, min_points=10):
    """
    Apply DBSCAN clustering to segment objects in the non-ground point cloud.
    eps: distance threshold
    min_points: minimum number of points to form a cluster
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
    return pcd, labels

def visualize_clusters(pcd, labels):
    """
    Visualize the clustered point cloud. Each cluster is shown in a different color.
    """
    max_label = labels.max()
    print(f"Detected {max_label + 1} clusters")

    # Assign random colors to each cluster
    colors = plt.get_cmap("tab20")(labels / (max_label + 1 if max_label > 0 else 1))
    colors[labels < 0] = [0, 0, 0, 1]  # Noise as black
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])  # RGB only
    o3d.visualization.draw_geometries([pcd], window_name="DBSCAN Object Clusters")



In [ ]:

# Cluster non-ground points
pcd, labels = cluster_dbscan(non_ground, eps=0.5, min_points=10)

# Visualize clusters
# each cluster=unique color, noise = black
visualize_clusters(pcd, labels)


Precompute neighbors.[================>           Detected 181 clusters
Precompute neighbors.[========================================] 100%


In [ ]:
#L-shape bounding box 

def fit_l_shape_box(cluster_points):
    """
    Fit an L-shape or oriented bounding box to a 2D-projected cluster (x, y).
    Returns center, angle, width, height of the rectangle.
    """
    if len(cluster_points) < 5:
        return None  # Not enough points

    points_2d = np.array(cluster_points)[:, :2].astype(np.float32)  # Project to (x, y)
    
    rect = cv2.minAreaRect(points_2d)
    box = cv2.boxPoints(rect)
    box = np.intp(box)

    return rect, box

def draw_l_shape_boxes(pcd, labels, eps=0.5, min_points=10):
    """
    Fit and draw L-shape boxes for each detected cluster.
    """
    clusters = {}
    for i in range(labels.max() + 1):
        clusters[i] = []

    points = np.asarray(pcd.points)
    for idx, label in enumerate(labels):
        if label != -1:  # Ignore noise
            clusters[label].append(points[idx])

    vis_cloud = pcd
    geometries = [vis_cloud]

    for cid, cluster_pts in clusters.items():
        cluster_pts = np.array(cluster_pts)
        result = fit_l_shape_box(cluster_pts)
        if result is None:
            continue
        rect, box = result

        # Create bounding box as line set for visualization
        lines = [
            [0, 1], [1, 2], [2, 3], [3, 0]
        ]
        colors = [[1, 0, 0] for _ in lines]  # Red lines
        line_set = o3d.geometry.LineSet()
        box_3d = np.c_[box, np.zeros(4)]  # Add z=0 for 3D visualization
        line_set.points = o3d.utility.Vector3dVector(box_3d)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.colors = o3d.utility.Vector3dVector(colors)
        geometries.append(line_set)

    o3d.visualization.draw_geometries(geometries, window_name="L-Shape Boxes on Clusters")



In [ ]:
# Apply L-shape fitting on previously clustered pcd and labels
# visualize each box with red edges

draw_l_shape_boxes(pcd, labels)
